In [5]:
## In-Class Assignment 7 - Evan Callaghan

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'Fish.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Using the pandas library to read the csv data file and create a data-frame called fish
fish = pd.read_csv(file_content_stream)

fish.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [6]:
## 2. Using the Length1, Lenght2, Length3, Height, and Width as the predictor variables, and Weight as the target variable to split the data into 
## train (80%) and test (20%)

## Defining input and target variables
X = fish[['Length1', 'Length2', 'Length3', 'Height', 'Width']]
Y = fish['Weight']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [8]:
## 3. Using the train dataset to perform LASSO as a variable selector

## Lasso CV for estimating alpha
lasso_cv = LassoCV(normalize = True, cv = 5, max_iter = 10000).fit(X_train, Y_train)

## Extracting the optimal alpha 
lasso_alpha = lasso_cv.alpha_

## Variable selection
lasso_md = Lasso(alpha = lasso_alpha, normalize = True, max_iter = 10000).fit(X_train, Y_train)

lasso_md.coef_

## Length2 set equal to zero! Can be eliminated from the test and train sets.

array([ 45.78588778,   0.        , -21.12867079,  21.94038193,
        36.66113032])

In [9]:
## 4. Using the variables from LASSO regression to normalize the inputs variables of the train and test datasets using the L2 normalization

## Dropping the Length2 variable
X_train = X_train.drop(columns = ['Length2'], axis = 1)
X_test = X_test.drop(columns = ['Length2'], axis = 1)


## Function for l2 normalization
def l2_normalization(X):
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    l2_norm = (X - x_mean) / l2
    return l2_norm


## Normalizing the train and test sets using the above function
X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)


In [13]:
## 5. Using the train dataset to build a linear regression model


lm_md1 = LinearRegression().fit(X_train, Y_train)

md1_pred = lm_md1.predict(X_test)

mse1 = np.mean(np.power(md1_pred - Y_test, 2))

## Reporting the MSE of this model.
mse1

## The MSE of this model is 108325

108325.40538082985

In [17]:
## 6.  Using the train dataset to build a ridge regression model

## Estimating alpha
ridge_cv = RidgeCV(alphas = [0.001, 0.01, 0.1, 1, 10, 100], cv = 5).fit(X_train, Y_train)

ridge_alpha = ridge_cv.alpha_

## Building the ridge model
ridge_md = Ridge(alpha = ridge_alpha).fit(X_train, Y_train)

## Predicting on the test set
md2_pred = ridge_md.predict(X_test)

## Calculating the MSE of ridge model
mse2 = np.mean(np.power(md2_pred - Y_test, 2))
mse2

## The MSE of this model is 106683

106683.294334641

In [ ]:
## 7. Using the results from parts (5) and (6), we would select the ridge regression model to predict fish weight because it has a smaller
## MSE value.